<a href="https://colab.research.google.com/github/rjroxx/NLP_Coursework/blob/master/TopicModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import zipfile
import nltk
from sklearn.decomposition import NMF,LatentDirichletAllocation 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer

In [55]:
nltk.download('punkt')
nltk.download('stopwords')
stw = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [56]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [57]:
!kaggle datasets download -d hj5992/nlp-topic-modelling

nlp-topic-modelling.zip: Skipping, found more recently modified local copy (use --force to force download)


In [58]:
datafolder = zipfile.ZipFile('/content/nlp-topic-modelling.zip')
datafolder.filelist

[<ZipInfo filename='Reviews.csv' compress_type=deflate file_size=300904694 compress_size=120062195>]

In [0]:
df = pd.read_csv(datafolder.open('Reviews.csv'),header=0)
df=df.head(500)

In [0]:
def word_preprocessor(text):
  text=re.sub('[^a-zA-Z]', " ", str(text))
  text=text.lower()
  text_tokens=word_tokenize(text)
  text_tokens_no_sw=[w for w in text_tokens if not w in stw]
  text=TreebankWordDetokenizer().detokenize(text_tokens_no_sw)
  return text

In [0]:
df['cleaned_text']=df['Text'].apply(word_preprocessor)

In [62]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleaned_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanuts p...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confection around centuries light pillowy citr...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...


In [0]:
df.drop(['Text'],1,inplace=True)

In [0]:
tf_idf=TfidfVectorizer()
score=tf_idf.fit_transform(df['cleaned_text'])
dense=score.todense()
dense1=dense.tolist()
col_names=tf_idf.get_feature_names()

In [0]:
df = pd.DataFrame(dense1,columns = col_names)

In [66]:
df.head()

,abby,abdominal,able,absence,absolute,absolutely,absorbs,abt,acai,accept,accompaniment,according,account,accumulated,acid,across,activate,active,activity,acts,actual,actually,ad,add,added,addict,addicted,addicting,addiction,addictive,addicts,adding,addition,additionally,additives,addled,adds,adjustments,admit,admittedly,...,wrap,wrapped,write,writing,wrong,wrote,wu,wuss,www,xdzik,xi,ya,yawning,yay,yeah,year,years,yelling,yerba,yes,yet,yi,yike,yoga,yogurt,york,youll,young,younger,youngest,yrs,yucky,yum,yummy,zack,zen,zero,zest,zing,zip
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.122363,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.274493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.209934,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
nmf=NMF(10,random_state=42,alpha=0.1,l1_ratio=0.5,init='nndsvd')
arr=nmf.fit_transform(df)
comps=nmf.components_

In [68]:
comps

array([[0.        , 0.        , 0.01061965, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
lda=LatentDirichletAllocation(n_components=10)
model1=lda.fit_transform(df)
comps1=lda.components_

In [0]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [71]:
print_topics(lda,tf_idf,100)


Topic #0:
cat cats food delivery however weight dogs products problems still love back old even felidae company candy past fast half flour home aisle price lunch get desserts advertised product store refund trouble mine happy three watchers reputation right less craving impossible health habit started heart packaging black expecting terms apple bound unable winter wife mostly individual retail savings wheatgrass sprouting rye rotate cut create older hunting melted animals fairly throw years issued thawed accept parents around must grass life reasonable berries stores weather filled let anyone locally helpful draw delivered fyi impression awesome dishwasher washed suggests website beans finding purchased

Topic #1:
like food one allergies good try love ginger tried br found eating best shipping flavor pancakes taste great fresh friends spots vinegar better add day cats oatmeal perfect bought worth soft really dog loved husband regularly eat juice coffee salt used without family time ca

In [72]:
print_topics(nmf,tf_idf,100)


Topic #0:
good like taste product love great one price get amazon coffee buy flavor really would use much better time bags store way order little try bag well eat box easy less go make mix stuff used tastes even chocolate could fresh find shipping bit grocery bad bought flavors still stores two add also local sure per tried free enough healthy found got cheaper think whole hard big size water candy brand many tasty thought ordering first perfect food flavored buying always every pretty take save delicious came products nice expensive online husband sometimes loved eating never fast without half small

Topic #1:
chips kettle potato flavor vinegar salt bag chip brand love favorite crunch cheddar great onion amount buy spicy crunchy eaten salty cooked really thai right perfect like bags eating never enjoy sweet flavors tangy thick tried greasy taste jalapeno many better perfectly dijon fat sour also oz recommend sea hard amazon honey friends eat amazing know open definitely top bit perso